# Creating a Convolutional Network with Keras and the CIFAR-10 Data Set

### Benjamin S. Knight

#### Start by downloading the data.

In [1]:
import os
from urllib.request import urlretrieve
if not os.path.isfile('cifar-10-python.tar.gz'):
    urlretrieve('https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', 'cifar-10-python.tar.gz')
else:
    print("The images are already downloaded.")

#### Let's unzip the files 

In [2]:
import tarfile
if not os.path.isfile('cifar-10-batches-py/batches.meta'):
    with tarfile.open('cifar-10-python.tar.gz') as tar:
        tar.extractall()
        tar.close()
else:
    print("The images are already unzipped.")

### The Training Data

#### Let's make these byte files human readable.

In [3]:
import pickle
batches = []

for i in range(1, 6, 1):
    with open('./cifar-10-batches-py/data_batch_' + str(i), mode='rb') as file:
        batches.append(pickle.load(file, encoding='latin1'))

#### Each batch is a dictionary with the following elements: A 10,000 element list of file names (e.g. 'american_elk_s_001521.png'), a 10,000 element list of labels ranging from 0-10, a 10,000x3071 array containing the image data. Each image is 32x32 for 1024 pixels. Each pixel has 3 channels (i.e. RBG), thus 32x32x3 = 3072.  Finally, there is the 'batch_label' - a 21 character string describing what batch the dictionary corresponds to (e.g. 'training batch 1 of 5').

In [4]:
example = batches[0]
for key in example.keys():
  print('The ' + key + ' element has a length of ' + str(len(example[str(key)])) + '.')

The batch_label element has a length of 21.
The labels element has a length of 10000.
The data element has a length of 10000.
The filenames element has a length of 10000.


#### Here is the image data. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue.

In [5]:
import pandas as pd
pd.DataFrame(example["data"]).head(5)

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,59,43,50,68,98,119,139,145,149,149,...,59,58,65,59,46,57,104,140,84,72
1,154,126,105,102,125,155,172,180,142,111,...,22,42,67,101,122,133,136,139,142,144
2,255,253,253,253,253,253,253,253,253,253,...,78,83,80,69,66,72,79,83,83,84
3,28,37,38,42,44,40,40,24,32,43,...,53,39,59,42,44,48,38,28,37,46
4,170,168,177,183,181,177,181,184,189,189,...,92,88,85,82,83,79,78,82,78,80


#### Here are the labels.

In [6]:
label_values = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 
                'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
pd.DataFrame(label_values, index=[0])

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,0,1,2,3,4,5,6,7,8,9


#### We can aggregate the labels into a single array. We use keras.utils for the one-hot-encoding.

In [7]:
from keras.utils import np_utils
labels = []   
for i in batches:
     labels.append(i['labels'])
from itertools import chain
y_train = list(chain.from_iterable(labels))  
y_train= np_utils.to_categorical(y_train, 10)
y_train.shape

Using TensorFlow backend.


(50000, 10)

In [8]:
data = [] 
for i in batches:
     data.append(pd.DataFrame(i['data']))
data = pd.concat(data) 
data = data.values
print("Our array of training features is " + str(len(data)) + " in length.")  

Our array of training features is 50000 in length.


In [9]:
x_train = data.reshape((len(data), 3, 32, 32)).transpose(0, 2, 3, 1)
x_train.shape

(50000, 32, 32, 3)

#### The Test Data 

In [10]:
with open('./cifar-10-batches-py/test_batch', mode='rb') as file:
    test_data = pickle.load(file, encoding='latin1')
y_test = test_data['labels']
y_test = np_utils.to_categorical(y_test, 10)
y_test.shape

(10000, 10)

In [11]:
x_test = test_data['data']
x_test = x_test.reshape((len(test_data['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
x_test.shape

(10000, 32, 32, 3)

### The Model

In [12]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import initializers

In [13]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3), 
          kernel_initializer=initializers.random_normal(stddev=0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(32, 3, 3,
          kernel_initializer=initializers.random_normal(stddev=0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3,
          kernel_initializer=initializers.random_normal(stddev=0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

/home/bsknight/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., kernel_initializer=<keras.ini...)`
  '` call to the Keras 2 API: ' + signature)
/home/bsknight/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer=<keras.ini...)`
  '` call to the Keras 2 API: ' + signature)
/home/bsknight/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_initializer=<keras.ini...)`
  '` call to the Keras 2 API: ' + signature)


In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [15]:
from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm
model.fit(
                   x_train 
                  ,y_train
                  ,epochs=1
                  ,batch_size=1024
                  ,verbose=0
                  ,callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)]
         )

In [16]:
score = model.evaluate(x_test, y_test, batch_size=1024, verbose=0)

In [17]:
print('The ' + str(model.metrics_names[0]) + ' is ' + str(score[0]) + '.')
print('The ' + str(model.metrics_names[1]) + ' is ' + str(score[1]) + '.')

The loss is 2.25843208504.
The acc is 0.184700000334.


What did classifications did our Keras model predict for the test set?

In [18]:
predictions = pd.DataFrame(model.predict_classes(x_test, batch_size=2048, verbose=0))
predictions[0].value_counts()

6    4443
0    4424
4     351
2     323
1     223
7     198
8      35
9       3
Name: 0, dtype: int64

These results are not even remotely correct. The training images and uniformly distributed.

In [19]:
pd.DataFrame(test_data['labels'])[0].value_counts()

7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
9    1000
1    1000
8    1000
0    1000
Name: 0, dtype: int64